In [45]:
import pandas as pd
import numpy as np
from IPython.display import display

In [2]:
# pass in column names for each CSV
# read_csv(..., sep='|') 

u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('../data/ml-100k/u.user', sep='|', names=u_cols,
                    encoding='latin-1')

In [3]:
users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [5]:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('../data/ml-100k/u.data', sep='\t', names=r_cols,
                      encoding='latin-1')

In [6]:
ratings.head()

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [7]:
# the movies file contains columns indicating the movie's genres
# let's only load the first five columns of the file with usecols
m_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url']
movies = pd.read_csv('../data/ml-100k/u.item', sep='|', names=m_cols, usecols=range(5),
                     encoding='latin-1')

In [8]:
movies.head()

,movie_id,title,release_date,video_release_date,imdb_url
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995)


### How to group data?

In [11]:
# read any file's first few lines
!head -n 3 ../data/city-of-chicago-salaries.csv

Name,Position Title,Department,Employee Annual Salary
"AARON,  ELVIA J",WATER RATE TAKER,WATER MGMNT,$85512.00
"AARON,  JEFFERY M",POLICE OFFICER,POLICE,$75372.00


In [38]:
# re-name columns while reading datafile
headers = ['name', 'title', 'department', 'salary']
chicago = pd.read_csv('../data/city-of-chicago-salaries.csv', 
                      header=0,
                      names=headers,
                      converters={'salary': lambda x: float(x.replace('$', ''))})
chicago.head()

,name,title,department,salary
0,"AARON, ELVIA J",WATER RATE TAKER,WATER MGMNT,85512.0
1,"AARON, JEFFERY M",POLICE OFFICER,POLICE,75372.0
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,80916.0
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,99648.0
4,"ABBATACOLA, ROBERT J",ELECTRICAL MECHANIC,AVIATION,89440.0


In [39]:
chicago.shape

(32054, 4)

In [15]:
from IPython.display import display

In [20]:
by_dept = chicago.groupby('department')
by_dept

In [21]:
print(by_dept.count().head()) # NOT NULL records within each column
print('\n')
print(by_dept.size().tail()) # total records for each department

                   name  title  salary
department                            
ADMIN HEARNG         42     42      42
ANIMAL CONTRL        61     61      61
AVIATION           1218   1218    1218
BOARD OF ELECTION   110    110     110
BOARD OF ETHICS       9      9       9


department
PUBLIC LIBRARY     926
STREETS & SAN     2070
TRANSPORTN        1168
TREASURER           25
WATER MGMNT       1857
dtype: int64


In [22]:
print(by_dept.sum()[20:25]) # total salaries of each department
print('\n')
print(by_dept.mean()[20:25]) # average salary of each department
print('\n')
print(by_dept.median()[20:25]) # take that, RDBMS!

                       salary
department                   
HUMAN RESOURCES     4850928.0
INSPECTOR GEN       4035150.0
IPRA                7006128.0
LAW                31883920.2
LICENSE APPL COMM     65436.0


                         salary
department                     
HUMAN RESOURCES    71337.176471
INSPECTOR GEN      80703.000000
IPRA               82425.035294
LAW                70853.156000
LICENSE APPL COMM  65436.000000


                    salary
department                
HUMAN RESOURCES    68496.0
INSPECTOR GEN      76116.0
IPRA               82524.0
LAW                66492.0
LICENSE APPL COMM  65436.0


In [24]:
# each group has how many unique titles
by_dept.title.nunique()

department
ADMIN HEARNG              15
ANIMAL CONTRL             19
AVIATION                 125
BOARD OF ELECTION         23
BOARD OF ETHICS            9
BUDGET & MGMT             24
BUILDINGS                 49
BUSINESS AFFAIRS          64
CITY CLERK                39
CITY COUNCIL              28
COMMUNITY DEVELOPMENT     71
CULTURAL AFFAIRS          39
DISABILITIES              22
DoIT                      40
FAMILY & SUPPORT         113
FINANCE                   98
FIRE                      91
GENERAL SERVICES         117
HEALTH                   118
HUMAN RELATIONS           12
HUMAN RESOURCES           32
INSPECTOR GEN             23
IPRA                      22
LAW                       50
LICENSE APPL COMM          1
MAYOR'S OFFICE            28
OEMC                      69
POLICE                   130
POLICE BOARD               2
PROCUREMENT               30
PUBLIC LIBRARY            72
STREETS & SAN             71
TRANSPORTN               150
TREASURER                 16
WAT

In [23]:
by_dept.title.nunique().sort_values(ascending=False)[:5]

department
WATER MGMNT    153
TRANSPORTN     150
POLICE         130
AVIATION       125
HEALTH         118
Name: title, dtype: int64

### split apply combine

In [25]:
def ranker(df):
    """Assigns a rank to each employee based on salary, with 1 being the highest paid.
    Assumes the data is DESC sorted."""
    df['dept_rank'] = np.arange(len(df)) + 1
    return df

In [29]:
chicago.shape

(32054, 5)

In [31]:
chicago.department.nunique()

35

In [32]:
len(by_dept)

35

In [33]:
chicago.sort_values('salary', ascending=False, inplace=True)

In [34]:
chicago.head()

,name,title,department,salary,dept_rank
18039,"MC CARTHY, GARRY F",SUPERINTENDENT OF POLICE,POLICE,260004.0,1
8004,"EMANUEL, RAHM",MAYOR,MAYOR'S OFFICE,216210.0,1
25588,"SANTIAGO, JOSE A",FIRE COMMISSIONER,FIRE,202728.0,1
27594,"STEWART III, CHARLES",FIRST DEPUTY FIRE COMMISSIONER,FIRE,188316.0,2
31587,"WYSINGER, ALFONZA",FIRST DEPUTY SUPERINTENDENT,POLICE,188316.0,2


In [40]:
chicago_ranker = chicago.groupby('department').apply(ranker)

In [41]:
chicago_ranker.shape

(32054, 5)

In [42]:
chicago_ranker.head()

,name,title,department,salary,dept_rank
0,"AARON, ELVIA J",WATER RATE TAKER,WATER MGMNT,85512.0,1
1,"AARON, JEFFERY M",POLICE OFFICER,POLICE,75372.0,1
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,80916.0,1
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,99648.0,2
4,"ABBATACOLA, ROBERT J",ELECTRICAL MECHANIC,AVIATION,89440.0,1


In [46]:
display(chicago_ranker[chicago_ranker.dept_rank == 1].head(14))

,name,title,department,salary,dept_rank
0,"AARON, ELVIA J",WATER RATE TAKER,WATER MGMNT,85512.0,1
1,"AARON, JEFFERY M",POLICE OFFICER,POLICE,75372.0,1
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,80916.0,1
4,"ABBATACOLA, ROBERT J",ELECTRICAL MECHANIC,AVIATION,89440.0,1
5,"ABBATEMARCO, JAMES J",FIREFIGHTER,FIRE,78012.0,1
7,"ABBOTT, BETTY L",FOSTER GRANDPARENT,FAMILY & SUPPORT,2756.0,1
9,"ABBRUZZESE, WILLIAM J",INVESTIGATOR - IPRA II,IPRA,62832.0,1
14,"ABDELQAWI, YOSEPH",TRAFFIC CONTROL AIDE-HOURLY,OEMC,19167.2,1
17,"ABDULLAH, ASKIA",LEGISLATIVE AIDE,CITY COUNCIL,25008.0,1
24,"ABERCROMBIE, TIMOTHY",MOTOR TRUCK DRIVER,STREETS & SAN,70408.0,1


In [47]:
chicago[chicago.department == "LAW"][:5]

,name,title,department,salary
76,"ACHILLES, AMBER L",ASST CORPORATION COUNSEL SUPVSR,LAW,91068.0
89,"ACUFF, KRISTIN L",ASST CORPORATION COUNSEL,LAW,66960.0
130,"ADAMS, OLIVIA I",LEGAL SECRETARY,LAW,72936.0
196,"AGUIAR, WILLIAM M",ASST CORPORATION COUNSEL SUPVSR,LAW,106416.0
273,"AKERS, AMY A",EXECUTIVE SECRETARY I,LAW,45684.0
